In [22]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.externals import joblib

from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy.sparse import hstack
from scipy.special import logit, expit

In [21]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../jigsaw-toxic-comment-classification-challenge/train.csv').fillna(' ')
test = pd.read_csv('../jigsaw-toxic-comment-classification-challenge/test.csv').fillna(' ')

list_sentences_train = train['comment_text']
list_sentences_test = test['comment_text']
all_text = pd.concat([list_sentences_train, list_sentences_test])

In [14]:
all_text

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
5         "\n\nCongratulations from me as well, use the ...
6              COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7         Your vandalism to the Matt Shirvington article...
8         Sorry if the word 'nonsense' was offensive to ...
9         alignment on this subject and which are contra...
10        "\nFair use rationale for Image:Wonju.jpg\n\nT...
11        bbq \n\nbe a man and lets discuss it-maybe ove...
12        Hey... what is it..\n@ | talk .\nWhat is it......
13        Before you start throwing accusations and warn...
14        Oh, and the girl above started her arguments w...
15        "\n\nJuelz Santanas Age\n\nIn 2002, Juelz Sant...
16        Bye! \n\nDon't look, come or t

# Clean Text

In [17]:
clean_word_dict

{'ａ': 'a',
 'ｂ': 'b',
 'ｃ': 'c',
 'ｄ': 'd',
 'ｅ': 'e',
 'ｆ': 'f',
 'ｇ': 'g',
 'ｈ': 'h',
 'ｉ': 'i',
 'ｊ': 'j',
 'ｋ': 'k',
 'ｌ': 'l',
 'ｍ': 'm',
 'ｎ': 'n',
 'ｏ': 'o',
 'ｐ': 'p',
 'ｑ': 'q',
 'ｒ': 'r',
 'ｓ': 's',
 'ｔ': 't',
 'ｕ': 'u',
 'ｖ': 'v',
 'ｗ': 'w',
 'ｘ': 'x',
 'ｙ': 'y',
 'ｚ': 'z',
 'Ａ': 'a',
 'Ｂ': 'b',
 'Ｃ': 'c',
 'Ｄ': 'd',
 'Ｅ': 'e',
 'Ｆ': 'f',
 'Ｇ': 'g',
 'Ｈ': 'h',
 'Ｉ': 'i',
 'Ｊ': 'j',
 'Ｋ': 'k',
 'Ｌ': 'l',
 'Ｍ': 'm',
 'Ｎ': 'n',
 'Ｏ': 'o',
 'Ｐ': 'p',
 'Ｑ': 'q',
 'Ｒ': 'r',
 'Ｓ': 's',
 'Ｔ': 't',
 'Ｕ': 'u',
 'Ｖ': 'v',
 'Ｗ': 'w',
 'Ｘ': 'x',
 'Ｙ': 'y',
 'Ｚ': 'z',
 '？': ' ?',
 '！': ' !',
 '．': ' .',
 '＝': ' =',
 '＋': ' +',
 '－': ' -',
 '１': '1',
 '２': '2',
 '３': '3',
 '４': '4',
 '５': '5',
 '６': '6',
 '７': '7',
 '８': '8',
 '９': '9',
 '０': '0'}

In [16]:
import re

cl_path = './cleaning/clean_letters.txt'
clean_word_dict = {}
with open(cl_path, 'r', encoding='utf-8') as cl:
    for line in cl:
        line = line.strip('\n')
        typo, correct = line.split(',')
        clean_word_dict[typo] = correct

def clean_word(text):
    replace_numbers = re.compile(r'\d+', re.IGNORECASE)
    special_character_removal = re.compile(r'[^a-z\d ]', re.IGNORECASE)

    text = text.lower()
    text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", text)
    text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", text)

    for typo, correct in clean_word_dict.items():
        text = re.sub(typo, " " + correct + " ", text)

    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"i’m", "i am", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = replace_numbers.sub('', text)
    return text

train_text = []
test_text = []
for text in list_sentences_train:
    train_text.append(clean_word(text))
    
for text in list_sentences_test:
    test_text.append(clean_word(text))

In [46]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=50000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     ngram_range=(1, 6),
#     max_features=30000)
# char_vectorizer.fit(all_text)

In [57]:
word_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
# train_char_features = char_vectorizer.transform(train_text)
# test_char_features = char_vectorizer.transform(test_text)

# Extra Tree Classifier

In [150]:
train_features = train_word_features
test_features = test_word_features
losses = []
predictions = {'id': test['id']}
model_dict = dict()
for class_name in class_names:
    train_target = train[class_name]
    classifier = ExtraTreesClassifier(n_estimators=30)
    
    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))
    
    classifier.fit(train_features, train_target)
    model_dict[class_name] = classifier
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

CV score for class toxic is 0.9563520283043244
CV score for class severe_toxic is 0.9374788299258515
CV score for class obscene is 0.9779372902677976
CV score for class threat is 0.8677795057582118
CV score for class insult is 0.9630752821396903
CV score for class identity_hate is 0.8979753961074951


In [233]:
# Load the model from the file 
model_dict_imported = joblib.load('models/models.p') 

In [225]:

def raw_chat_to_model_input(raw_input_string):
    
    cleaned_text = []
    for text in [raw_input_string]:
        cleaned_text.append(clean_word(text))
    #print(cleaned_text)
    return word_vectorizer.transform(cleaned_text)

    
def predict_toxicity(raw_input_string):
    model_input = raw_chat_to_model_input(raw_input_string)
    results = []
    for key,model in model_dict.items():
        results.append(round(model.predict_proba(model_input)[0,1],4))
    return results

In [163]:
raw_chat_to_model_input(chat_input)

['fuck you this is bullshit gay queer']


<1x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [164]:
model_dict['toxic'].predict_proba(raw_chat_to_model_input(chat_input))

['fuck you this is bullshit gay queer']


array([[0.06666667, 0.93333333]])

In [189]:
len(output_list[0])

6

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [250]:

chat_input = 'trash is garbage'

output_list = [list(model_dict.keys()),predict_toxicity(chat_input)]
for index in range(len(output_list[0])):
    print(output_list[0][index],output_list[1][index])

toxic 0.7
severe_toxic 0.1333
obscene 0.0703
threat 0.0
insult 0.0685
identity_hate 0.1667


In [118]:
for key in model_dict:
    print(key)

toxic
severe_toxic
obscene
threat
insult
identity_hate


In [221]:
forsen_chat = joblib.load('./chat_logs/forsen_chat.p')

In [231]:
forsen_chat.shape

(14019, 4)

In [232]:
forsen_chat['message'][0:1000].apply(lambda msg : predict_toxicity(msg))

0            [0.0, 0.0, 0.0333, 0.0, 0.0, 0.0]
1         [0.027, 0.0, 0.027, 0.0, 0.027, 0.0]
2         [0.027, 0.0, 0.027, 0.0, 0.027, 0.0]
3         [0.0, 0.0, 0.0333, 0.0, 0.0333, 0.0]
4            [0.0, 0.0, 0.0333, 0.0, 0.0, 0.0]
5         [0.0333, 0.0, 0.0333, 0.0, 0.0, 0.0]
6         [0.0, 0.0, 0.0333, 0.0, 0.0333, 0.0]
7      [0.1333, 0.0, 0.0333, 0.0, 0.0667, 0.0]
8         [0.091, 0.0, 0.091, 0.0, 0.027, 0.0]
9         [0.0, 0.0, 0.0333, 0.0, 0.0333, 0.0]
10     [0.2333, 0.0, 0.0333, 0.0333, 0.0, 0.0]
11           [0.0, 0.0, 0.0333, 0.0, 0.0, 0.0]
12        [0.0, 0.0, 0.0333, 0.0, 0.0333, 0.0]
13        [0.091, 0.0, 0.091, 0.0, 0.027, 0.0]
14       [0.027, 0.0, 0.0919, 0.0, 0.027, 0.0]
15           [0.0, 0.0, 0.0333, 0.0, 0.0, 0.0]
16        [0.0, 0.0, 0.0333, 0.0, 0.0333, 0.0]
17           [0.0, 0.0, 0.0333, 0.0, 0.0, 0.0]
18        [0.0333, 0.0, 0.0333, 0.0, 0.0, 0.0]
19     [0.2333, 0.0, 0.0333, 0.0333, 0.0, 0.0]
20     [0.0333, 0.0, 0.0, 0.0, 0.0333, 0.1333]
21        [0.